In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
# import csv
full_data = pd.read_csv("mbti_1.csv")

# selecting random percentage of rows because of memory issues
data = full_data.sample(frac = 0.2)
data.columns = ['type', 'posts']

In [3]:
import string
#calculating the average post length
data['avg_post_len'] = data['posts'].apply(lambda x: (len(x) - x.count(" "))/50)
data

,type,posts,avg_post_len
5145,INTJ,'1. Do you guys subscribe to holidays e.g. 4th...,107.84
7098,ENFP,'Very interesting test. Found some of the ques...,140.62
3358,INFP,'Aw thank you ☺️☺️ It's of the Shedd Aquarium ...,105.18
5518,ENTP,'I'm quite sure that an ENTP can spend lots of...,123.38
1981,ISTJ,'http://4.bp.blogspot.com/-9QjARWd5a84/T3OXHJJ...,107.44
...,...,...,...
6421,ENTP,'I dislike being inconsistent. I also dislike ...,130.82
2208,INFJ,'Whatever. :p|||;)|||I can't ok? That's kind o...,115.96
5412,INTJ,':toast: likewise (though I already know lots ...,143.26
6379,INFJ,'Sensors r so dum lolol da N label valid8 my ...,103.62


In [4]:
#calculating the total post length
data['tot_post_len'] = data['posts'].apply(lambda x: len(x) - x.count(" "))
data

,type,posts,avg_post_len,tot_post_len
5145,INTJ,'1. Do you guys subscribe to holidays e.g. 4th...,107.84,5392
7098,ENFP,'Very interesting test. Found some of the ques...,140.62,7031
3358,INFP,'Aw thank you ☺️☺️ It's of the Shedd Aquarium ...,105.18,5259
5518,ENTP,'I'm quite sure that an ENTP can spend lots of...,123.38,6169
1981,ISTJ,'http://4.bp.blogspot.com/-9QjARWd5a84/T3OXHJJ...,107.44,5372
...,...,...,...,...
6421,ENTP,'I dislike being inconsistent. I also dislike ...,130.82,6541
2208,INFJ,'Whatever. :p|||;)|||I can't ok? That's kind o...,115.96,5798
5412,INTJ,':toast: likewise (though I already know lots ...,143.26,7163
6379,INFJ,'Sensors r so dum lolol da N label valid8 my ...,103.62,5181


In [ ]:
# calculating the punctuation percentage
# def punct_count(post):
#     count = sum([1 for char in post if char in string.punctuation])
#     return round(count/(len(post) - post.count(" ")), 3)*100

# data['punct_%'] = data['posts'].apply(lambda x: punct_count(x))
# data

In [5]:
data.reset_index(drop=True, inplace=True)
data

,type,posts,avg_post_len,tot_post_len
0,INTJ,'1. Do you guys subscribe to holidays e.g. 4th...,107.84,5392
1,ENFP,'Very interesting test. Found some of the ques...,140.62,7031
2,INFP,'Aw thank you ☺️☺️ It's of the Shedd Aquarium ...,105.18,5259
3,ENTP,'I'm quite sure that an ENTP can spend lots of...,123.38,6169
4,ISTJ,'http://4.bp.blogspot.com/-9QjARWd5a84/T3OXHJJ...,107.44,5372
...,...,...,...,...
1730,ENTP,'I dislike being inconsistent. I also dislike ...,130.82,6541
1731,INFJ,'Whatever. :p|||;)|||I can't ok? That's kind o...,115.96,5798
1732,INTJ,':toast: likewise (though I already know lots ...,143.26,7163
1733,INFJ,'Sensors r so dum lolol da N label valid8 my ...,103.62,5181


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [7]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
stopwords =nltk.corpus.stopwords.words('english')
from nltk.stem import PorterStemmer
ps = nltk.PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [13]:
# create function to clean the posts
def clean_posts(post):
    post = "".join([word.lower()for word in post if word not in string.punctuation])
    tokens = re.split('\W+', post)
    post = [ps.stem(word) for word in tokens if word not in stopwords]
    return post

#TF-IDF
# tf_vectorize = TfidfVectorizer(analyzer=clean_posts)
# X_tf = tf_vectorize.fit_transform(data['posts'])
# X_tf_feature = pd.concat([data['avg_post_len'], data['punct_%'], pd.DataFrame(X_tf.toarray())], axis=1)

# #Count Vectorizer
count_vectorize = CountVectorizer(analyzer = clean_posts)
X_count = count_vectorize.fit_transform(data['posts'])
X_count_save = np.array(X_count)
# X_count_feature = pd.concat([data['avg_post_len'], data['punct_%'], pd.DataFrame(X_count.toarray())], axis=1)
X_count_feature = pd.DataFrame(X_count.toarray())
X_count_feature.head()
X_count_save

array(<1735x90246 sparse matrix of type '<class 'numpy.int64'>'
	with 720405 stored elements in Compressed Sparse Row format>,
      dtype=object)

In [14]:
X_count_feature

,0,1,2,3,4,5,6,7,8,9,...,90236,90237,90238,90239,90240,90241,90242,90243,90244,90245
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X_count

<1735x90246 sparse matrix of type '<class 'numpy.int64'>'
	with 720405 stored elements in Compressed Sparse Row format>

NameError: name 'post' is not defined

In [ ]:
target = data['type']

In [ ]:
target

In [ ]:
X_count_feature

In [ ]:
rf=RandomForestClassifier()
param={'n_estimators':[400],'max_depth':[None]}

gs=GridSearchCV(rf,param,cv=3,n_jobs=-1)
gs_fit=gs.fit(X_count_feature,data['type'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False).head()

In [ ]:
gs = joblib.load("../count_vect_model.sav")

In [ ]:
test = pd.DataFrame({"posts":["I think I can I think I can I am not sure"]})

test

In [ ]:
import pickle

a = pd.DataFrame()
b = "this can be your vectorizer thing"

In [ ]:
def clean_posts(post):
    post = "".join([word.lower()for word in post if word not in string.punctuation])
    tokens = re.split('\W+', post)
    post = [ps.stem(word) for word in tokens if word not in stopwords]
    return post


# #Count Vectorizer
count_vectorize = CountVectorizer(analyzer = clean_posts)
X_count = count_vectorize.fit_transform(test['posts'])
X_count_feature = pd.DataFrame(X_count.toarray())

X_count_feature.head()

In [ ]:
gs.predict(X_count_feature)

In [ ]:
# yo uname the file here
with open('picklefile.pickle', 'wb') as f:
    pickle.dump([a, b], f)

In [ ]:
with open('picklefile.pickle', 'rb') as f:
    loaded_vars = pickle.load(f)

In [ ]:
loaded_vars

In [ ]:
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False).head()